In [1]:
#need to refine on the output of RDA_hovmoller by allowing us to create frequency diagrams for arbitrary time periods.
#also, importantly, now shows Hovmoller plot of intensity for idfferent time periods.

In [1]:
import numpy as np
import time
import os
import netCDF4 as nc
import datetime
import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
#Access NetCDF files (currently hosted in Ferret directory)
RDA_path_1 = "/Users/Siwen/Desktop/ferret/bin/meiyu_clean.nc"
RDA_path_2 = "/Users/Siwen/Desktop/ferret/bin/meiyu_2_clean.nc"
RDA_1 = nc.Dataset(RDA_path_1, 'r')
RDA_2 = nc.Dataset(RDA_path_2, 'r')

In [3]:
#load data from NetCDF files to notebook
lat_1_all =  RDA_1.variables['lat_115'][:]
lat_2_all =  RDA_2.variables['lat_115'][:]
intensity_1_all = RDA_1.variables['intensity'][:] 
intensity_2_all = RDA_2.variables['intensity'][:]

In [4]:
#Assign a calendar date to each time point
startday = datetime.datetime(1951,1,1)
date_list = np.array([datetime.timedelta(days=x) + startday for x in range(0, 20819)])

In [14]:
#Define empty variables reshaped with axis 0 as day of year and axis 1 as year.

#time periods of interest - period1 (p1) and period2 (p2)
#adjustment: mymap (below) now also restricts by years of interest.
p1_yr_b = [1951,1980]
p1_yr_e = [1979,1993]
p2_yr_b = [1980,1994]
p2_yr_e = [2007,2007]

#variable storage for period 1 and period 2 - initialized as e
lat_1_p1 = []
lat_2_p1 = []
intensity_1_p1 = []
intensity_2_p1 = []

lat_1_p2 = []
lat_2_p2 = []
intensity_1_p2 = []
intensity_2_p2 = []

## TIME PERIOD 1 - arrange data by day of year and then year
for p1_b, p1_e in zip(p1_yr_b, p1_yr_e):

    lat_1 = np.zeros([365, p1_e-p1_b+1])
    lat_2 = np.zeros([365, p1_e-p1_b+1])
    int_1 = np.zeros([365, p1_e-p1_b+1])
    int_2 = np.zeros([365, p1_e-p1_b+1])
        
    for i, date in enumerate(date_list[0:365]): #go through each day of the year
    
        dd = date.day
        mm = date.month
        mymap = map(lambda date: (date.day == dd) & (date.month == mm) & (date.year >= p1_b) & (date.year <= p1_e), date_list)
        extract = np.array(list(mymap))
    
        lat_1[i,:] = lat_1_all[extract]
        lat_2[i,:] = lat_2_all[extract]
        int_1[i,:] = intensity_1_all[extract]
        int_2[i,:] = intensity_2_all[extract]
                    
    lat_1_p1.append(lat_1)
    lat_2_p1.append(lat_2)
    intensity_1_p1.append(int_1)
    intensity_2_p1.append(int_2)
    
## TIME PERIOD 2    
for p2_b, p2_e in zip(p2_yr_b, p2_yr_e):

    lat_1 = np.zeros([365, p2_e-p2_b+1])
    lat_2 = np.zeros([365, p2_e-p2_b+1])
    int_1 = np.zeros([365, p2_e-p2_b+1])
    int_2 = np.zeros([365, p2_e-p2_b+1])
        
    for i, date in enumerate(date_list[0:365]): #go through each day of the year
    
        dd = date.day
        mm = date.month
        mymap = map(lambda date: (date.day == dd) & (date.month == mm) & (date.year >= p2_b) & (date.year <= p2_e), date_list)
        extract = np.array(list(mymap))
    
        lat_1[i,:] = lat_1_all[extract]
        lat_2[i,:] = lat_2_all[extract]
        int_1[i,:] = intensity_1_all[extract]
        int_2[i,:] = intensity_2_all[extract]
                    
    lat_1_p2.append(lat_1)
    lat_2_p2.append(lat_2)
    intensity_1_p2.append(int_1)
    intensity_2_p2.append(int_2)

In [17]:
lat_1_p1[0].shape

(365, 29)

In [ ]:
#convert latitude information into binning by latitude
#BINS: <20, 20-40 by 1 degree bin, and 40+
lat_1_index = np.round(lat_1-19.5)
lat_1_index[lat_1_index < 0] = 0
lat_1_index[lat_1_index > 21] = 21

lat_2_index = np.round(lat_2-19.5)
lat_2_index[lat_2_index < 0] = 0
lat_2_index[lat_2_index > 21] = 21

In [ ]:
#convert indices into Hovmoller plot
lat_1_hov = np.zeros([365,22])
lat_2_hov = np.zeros([365,22])

#below, we divide by 57 (the number of years in our data set) to switch units from raw counts to frequency.

#for primary fronts:
for dd, col in enumerate(lat_1_index):
    for ix in range(22):
        lat_1_hov[dd,ix] = np.count_nonzero(col == ix)/57
        
#for secondary fronts:
for dd, col in enumerate(lat_2_index):
    for ix in range(22):
        lat_2_hov[dd,ix] = np.count_nonzero(col == ix)/57

In [ ]:
#combine into single variable
lat_hov = lat_1_hov + lat_2_hov

In [ ]:
#handy smoothing function which allows for different smoothing in time direction and latitude direction, and also
#wraps time axis
def smooth(P,dayrange,latrange):
    Psmooth=np.zeros(P.shape)
    
    ll=int((dayrange-1)/2)
    yy=int((latrange-1)/2)
    
    days = P.shape[0]
    lats = P.shape[1]
    
    for d in range(days):
        for j in range(lats):
            
            Psample = P.take(range(d-ll,d+ll+1), mode='wrap', axis=0)
            Ps = Psample[:,max(0,j-yy):min(days,j+yy+1)]
            Psmooth[d,j] = np.mean(np.mean(Ps))
    
    return Psmooth

In [ ]:
#smoothed occupancy plots for frontal events
lat_1_hov_smth = smooth(lat_1_hov,15,5)
lat_2_hov_smth = smooth(lat_2_hov,15,5)
lat_hov_smth = smooth(lat_hov,15,5)

In [ ]:
#SAVE AS NETCDF FILE
savefile = "RDA_hov_"
ferretpath = "/Users/Siwen/Desktop/Ferret/bin/"

#netCDF output file formatting formatting
hovname = ferretpath + savefile + ".nc"

##the following is only to make sure that we don't exceed the bounds of the number of rows of actual data.
if os.path.isfile(hovname):
    os.remove(hovname)

hovout = nc.Dataset(hovname, "w")

#create dimensions
days_dim = hovout.createDimension('time', None)
lat_dim = hovout.createDimension('lat', 22)

dim_tuple = ("time","lat")

#create variables inside netCDF file
lat_1_out = hovout.createVariable("lat_1_hov", 'f8', dim_tuple)
lat_2_out = hovout.createVariable("lat_2_hov", 'f8', dim_tuple)
lat_out = hovout.createVariable("lat_hov", 'f8', dim_tuple)

lat_1_smth_out = hovout.createVariable("lat_1_hov_smth", 'f8', dim_tuple)
lat_2_smth_out = hovout.createVariable("lat_2_hov_smth", 'f8', dim_tuple)
lat_smth_out = hovout.createVariable("lat_hov_smth", 'f8', dim_tuple)

lat_1_out[:] = lat_1_hov
lat_2_out[:] = lat_2_hov
lat_out[:] = lat_hov

lat_1_smth_out[:] = lat_1_hov_smth
lat_2_smth_out[:] = lat_2_hov_smth
lat_smth_out[:] = lat_hov_smth

hovout.close()